In [ ]:
# Detection probability map to tiff stacks 

In [ ]:
import numpy as np
import json
from renderapps.module.render_module import RenderModule
import renderapi
import os
import synaptogram
import matplotlib.pyplot as plt
from scipy import misc
from skimage import measure


In [ ]:
args={
    'render':{
    'host':'http://152.3.214.113',
    'port':8080,
    'owner':'Anish',
    'project':'M247514_Rorb_1',
    'client_scripts':'/home/anish/Connectome/render/render-ws-java-client/src/main/scripts'
},
'log_level':'DEBUG'}
mod = RenderModule(input_data = args,args=[])
stacks = mod.render.run(renderapi.render.get_stacks_by_owner_project)
render_args = args['render']

In [ ]:
thresh = 0.9
resultVol = np.load('resultVol.npy')

In [ ]:
reload(synaptogram)

In [ ]:
resultVol.shape

In [ ]:
labelVol = measure.label(resultVol > thresh)
stats = measure.regionprops(labelVol, resultVol)

In [ ]:
min0 = 0
min1 = 0
min2 = 0 

for detection in stats: 
    for pt in detection.coords: 
        
        if pt[0] > min0:
            min0 = pt[0]
        
        if pt[1] > min1: 
            min1 = pt[1] 
            
        if pt[2] > min2: 
            min2 = pt[2]
            

In [ ]:
print [min0, min1, min2] # ROW(Y) COL(X) Z

In [ ]:
detection = stats[4000]

In [ ]:
detection.coords

In [ ]:
len(detection.coords)

In [ ]:
bbox = synaptogram.getBoundingBoxFromLabel(detection)

In [ ]:
bbox

In [ ]:
reload(synaptogram)

In [ ]:
labelvol = np.zeros((441, 420, 50))

In [ ]:
len(stats)

In [ ]:
bbox['maxZ']

In [ ]:
for detection in stats: 
    
    bbox = synaptogram.getBoundingBoxFromLabel(detection)
    zInds = synaptogram.getZListFromBoundingBox(bbox)
    
    for n in range(0, len(zInds)): 
        zInd = int(zInds[n])
        
        maxX = int(np.round(bbox['maxX']))
        minX = int(np.round(bbox['minX']))
        
        maxY = int(np.round(bbox['maxY']))
        minY = int(np.round(bbox['minY']))
        
        labelvol[minY:maxY, minX:maxX, zInd] = 1


In [ ]:
%matplotlib notebook

plt.imshow(labelvol[:, :, 46], cmap='gray')

In [ ]:
folderpath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/results'

In [ ]:
fn = os.path.join(folderpath, str(zInd).zfill(5))
fn = fn + '.tiff'

In [ ]:
fn

In [ ]:
for z in range(0, labelvol.shape[2]): 
    fn = os.path.join(folderpath, str(z).zfill(5))
    fn = fn + '.tiff'
    misc.imsave(fn, labelvol[:, :, z])
